# SurvivalStan

## Set up environment

In [1]:
# Install Stan utilities

!rm stan_utility.py
!wget https://raw.githubusercontent.com/betanalpha/jupyter_case_studies/master/pystan_workflow/stan_utility.py
    

# Upgrade pip
!pip3 install --user --upgrade pip


# Install PyStan
!pip3 install --user pystan


# Install SurvivalStan
!git clone https://github.com/hammerlab/survivalstan.git
!pip3 uninstall --yes survivalstan
!pip3 install --user survivalstan/.

--2019-07-24 05:26:34--  https://raw.githubusercontent.com/betanalpha/jupyter_case_studies/master/pystan_workflow/stan_utility.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.112.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5593 (5.5K) [text/plain]
Saving to: ‘stan_utility.py’

stan_utility.py     100%[===================>]   5.46K  --.-KB/s    in 0s      

2019-07-24 05:26:34 (61.8 MB/s) - ‘stan_utility.py’ saved [5593/5593]



In [ ]:
import sys
import os

import numpy as np
import pandas as pd


import pystan
import survivalstan

## Simple exponential model with covariates

This serves as a simple sanity check.

$S(t) = \exp( -at ) \exp( \beta \cdot X )$

The following prior strategy will be used:

* Priors for established risk factors
* Priors for factor with more uncertain relevance

In [5]:
# Simulate Data
df2 = survivalstan.sim.sim_data_exp_correlated(N = 500, 
                                               censor_time = 20, 
                                               rate_form = '1 + age + sex', 
                                               rate_coefs=[0, -3, 0.5])

# Plot data
survivalstan.utils.plot_observed_survival(
    df2.query('sex == "female"'),
    time_col='t',
    event_col='event',
    label='female')

survivalstan.utils.plot_observed_survival(
    df2.query('sex == "male"'),
    time_col='t',
    event_col='event',
    label='male')

plt.legend()

In [ ]:
# Infer parameters

ctrl = dict(adapt_delta = 0.99999,
            max_treedepth = 20)

fit2_org = survivalstan.fit_stan_survival_model(
    df=df2,
    time_col='t',
    event_col='event',
    model_code=survivalstan.models.exp_survival_model,
    formula='~ 1 + age + sex',
    model_cohort = 'exp model',
    iter = 10000,
    chains = 4,
    warmup = 6500,
    control = ctrl)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0176fed484efbd961d78e8973c25ff98 NOW.


NOT reusing model.


In [ ]:
# HMC diagnostics

sfit2_org = fit2_org['fit']

pystan.check_hmc_diagnostics(sfit2_org)

print(sfit2_org)

## More flexible baseline hazards: Piecewise hazard